In [1]:
import os
import sys
import pandas as pd
sys.path.append('/home/dane/src/datatools')

In [2]:
%cd /home/dane/src/datatools/ipy/notebooks/regtest/dev
df = pd.read_csv('../data/v14_b8600_b8658.csv')

/home/dane/src/datatools/ipy/notebooks/regtest/dev


In [3]:
from regtest.ingest_log_files import StrMap
pmap = StrMap.read('../../../../regtest/project_name')

# build a reverse map and shorten names to useful part
rmap = {v:k.replace('\\', '/')[37:] for k, v in pmap.items()}

In [4]:
df.head(1)

,passed,build,revision,project,machine,sim_time,elapsed_time,os,aut
0,1,B8622,REV10,0,TESTEXAMPLES,1.284,1.45576,0,153


In [5]:
def std_groupby(df):
    results = []
    for project, runs in df.groupby('project'):
        sim_mean = runs.sim_time.mean()
        std = runs.sim_time.std()
        sim_rsd = std / sim_mean
        outliers = len(runs[runs.sim_time > (sim_mean + 2 * std)])
        el_mean = runs.elapsed_time.mean()
        el_rsd = runs.elapsed_time.std() / el_mean
        results.append( (project, sim_mean, sim_rsd, outliers, el_mean, el_rsd) )

    test_times = pd.DataFrame.from_records(results,
                                           columns=['project', 'sim_mean', 'sim_rsd',
                                                    'outliers', 'el_mean', 'el_rsd'])


In [6]:
%timeit std_groupby(df)

1 loop, best of 3: 15 s per loop


In [7]:
from time import time
from multiprocessing import Pool

In [8]:
by_proj = df.sort('project')

/home/dane/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [9]:
def split_df(df, num):
    total_len = len(df)
    df_start_pos = 0
    new_len = int(total_len / num)
    newdfs = []
    bkpt = df_start_pos + new_len

    while bkpt < total_len:
        curproj = df.iloc[bkpt].project
        while df.iloc[bkpt].project == curproj:
            bkpt += 1
        print('creating df from {}:{}'.format(df_start_pos, bkpt - 1))
        newdfs.append(df.iloc[df_start_pos:(bkpt - 1)])
        df_start_pos = bkpt
        bkpt += new_len

    newdfs.append(df.iloc[df_start_pos:])

    print('returning {} dataframes of the following sizes:'.format(len(newdfs)))
    for x in newdfs:
        print('    ', len(x))

    return newdfs

def np(df):
    results = []
    print('in np with dataframe of size', len(df))
    for project, runs in df.groupby('project'):
        sim_mean = runs.sim_time.mean()
        std = runs.sim_time.std()
        sim_rsd = std / sim_mean
        outliers = len(runs[runs.sim_time > (sim_mean + 2 * std)])
        el_mean = runs.elapsed_time.mean()
        el_rsd = runs.elapsed_time.std() / el_mean
        results.append( (project, sim_mean, sim_rsd, outliers, el_mean, el_rsd) )

    test_times = pd.DataFrame.from_records(results,
                                           columns=['project', 'sim_mean', 'sim_rsd',
                                                    'outliers', 'el_mean', 'el_rsd'])
    return test_times


def summarize_pool(args):
    project, runs = args
    sim_mean = runs.sim_time.mean()
    std = runs.sim_time.std()
    sim_rsd = std / sim_mean
    outliers = len(runs[runs.sim_time > (sim_mean + 2 * std)])
    el_mean = runs.elapsed_time.mean()
    el_rsd = runs.elapsed_time.std() / el_mean
    return project, sim_mean, sim_rsd, outliers, el_mean, el_rsd


def p(num):
    global by_proj
    t0 = time()
    dfs = split_df(by_proj, num)
    pool = Pool(processes=num)
    results = pool.map(np, dfs)

    print('{:4.1f} s.'.format(time() - t0))

In [11]:
p(4)

creating df from 0:1706556
creating df from 1706557:3413178
creating df from 3413179:5119302
returning 4 dataframes of the following sizes:
     1706556
     1706621
     1706123
     1704678
in np with dataframe of size 1706556
in np with dataframe of size 1706621
in np with dataframe of size 1706123
in np with dataframe of size 1704678
11.4 s.
